In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
Save_to_html = True

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_names=["../VTXalignment/input/my-m_ee_Run14AuAu_180th_new_20140_1071runs.root","../sim/output/fall25/single/ccbar_soft_new_v59.root" ,"../sim/output/fall25/single/conv_v33.root","../sim/output/fall25/single/dalitz_v01.root" ] 
#"../VTXalignment/input/my-m_ee_Run14AuAu_107th_new_19943_1065runs.root"
names_flies = ["data","ccbar","conversion","dalitz"]
names_conv_phi = ["layer 2-1 0%-40%","layer 3-1 0%-40%","layer 3-2 0%-40%","layer 2-1 60%-90%", "layer 3-1 60%-90%","layer 3-2 60%-90%"]
names_dalitz_phi = ["layer 1-0 0%-40%","layer 2-0 0%-40%","layer 3-0 0%-40%","layer 2-1 0%-40%","layer 3-1 0%-40%","layer 3-2 0%-40%","layer 1-0 60%-90%","layer 2-0 60%-90%","layer 3-0 60%-90%","layer 2-1 60%-90%","layer 3-1 60%-90%","layer 3-2 60%-90%"]
names_sdphi = ["sdphi 0%-20%","sdphi 20%-40%","sdphi 40%-60%","sdphi 60%-80%","sdphi 80%-93%"]
names_sdphi0 = ["sdphi 0%-20%","sdphi 20%-40%","sdphi 40%-60%","sdphi 60%-80%","sdphi 80%-93%"]
names_isconv = ["0%-20%","20%-40%","40%-60%","60%-80%","80%-93%"]
hist_dalt_phi = [f"chi2_ndf_{i}" for i in range(5)]
hist_sdphi = [f"chi2npe0_hist_el"]
hist_select_3D_names = [hist_dalt_phi, hist_sdphi]
all_names = [names_conv_phi,names_dalitz_phi, names_sdphi, names_sdphi0, names_isconv]

In [4]:
N_hists = len(hist_select_3D_names)

In [5]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    #print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hist_read_hists = []
        for jhist in range(len(hist_select_3D_names[ihist])):
            hist_read_hists.append(infile.Get(hist_select_3D_names[ihist][jhist]))
            hist_read_hists[-1].SetDirectory(root.nullptr)
            hist_read_hists[-1].SetName(hist_read_hists[-1].GetName()+f"{iFile}")
        hists_read_file.append(hist_read_hists)
    hists_read.append(hists_read_file)


infile.Close()

In [6]:
pt_min, pt_max = 0.2, 5.
iFile = 0
type_loc = 0
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[iFile][type_loc][icentr].Project3D("xy")
    root.gPad.SetLogz()
    pos_proj.Draw("colz")

c0.Draw()

In [7]:
pt_min, pt_max = 0.2, 5.
iFile = 0
type_loc = 0
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[iFile][type_loc][icentr].ProjectionX(f"pos_prosjX{icentr}",21,21, ibinz0, ibinz1)
    root.gPad.SetLogz()
    pos_proj.Draw("colz")

c0.Draw()

In [8]:
pt_min, pt_max = 0.2, 5.
iFile = 0
type_loc = 0
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
legends.append(root.TLegend(0.6,0.7,0.9,0.9))
Format_Legend(legends[-1])
tot_int_0 = 0
root.gPad.SetLogy()
for iFile in range(0,4):
    for icentr in range(2,3):
        ibinz0 = projz.FindBin(pt_min)
        ibinz1 = projz.FindBin(pt_max)-1
        pos_proj = hists_read[iFile][type_loc][icentr].ProjectionX(f"pos_prosjX{icentr}_{iFile}",21,21, ibinz0, ibinz1)
        root.gPad.SetLogz()
        if iFile==0: 
            Format_Hist_total(pos_proj, Lcolor=colors[iFile],Lwidth=4, title_x="#chi^{2}/n.d.f. [GeV]", title_y="Counts")
            pos_proj.SetMaximum(12*pos_proj.GetMaximum())
            tot_int_0 = pos_proj.GetBinContent(1)
            pos_proj.Draw("")
        else: 
            pos_proj.Scale(tot_int_0/pos_proj.GetBinContent(1))
            Format_Graph(pos_proj, Lcolor=colors[iFile],Lwidth=4)
            pos_proj.Draw("same HIST")

        frac = pos_proj.Integral(pos_proj.FindBin(0.1), pos_proj.FindBin(0.9))/pos_proj.Integral()
        legends[-1].AddEntry(pos_proj, names_flies[iFile]+f" {frac:.2f}", "l")
legends[-1].Draw()
c0.Draw()
#c0.SaveAs("output/ml/ml_conv_dalitz_chi2_ndf_comp_pt02_5.pdf")

In [9]:
pt_min, pt_max = 76,125
iFile = 0
type_loc = 1
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
legends.append(root.TLegend(0.6,0.7,0.9,0.9))
Format_Legend(legends[-1])
tot_int_0 = 0
root.gPad.SetLogy()
for iFile in range(0,4):
    for icentr in range(0,1):
        ibinz0 = projz.FindBin(pt_min)
        ibinz1 = projz.FindBin(pt_max)-1
        pos_proj = hists_read[iFile][type_loc][icentr].ProjectionX(f"pos_prosjX{icentr}_{iFile}",8,45, ibinz0, ibinz1)
        root.gPad.SetLogz()
        if iFile==0: 
            Format_Hist_total(pos_proj, Lcolor=colors[iFile],Lwidth=4, title_x="#chi^{2}/n.d.f. [GeV]", title_y="Counts")
            pos_proj.SetMaximum(12*pos_proj.GetMaximum())
            tot_int_0 = pos_proj.Integral()
            pos_proj.Draw("")
        else: 
            pos_proj.Scale(tot_int_0/pos_proj.Integral())
            Format_Graph(pos_proj, Lcolor=colors[iFile],Lwidth=4)
            pos_proj.Draw("same HIST")
        frac = pos_proj.Integral(pos_proj.FindBin(2.85), pos_proj.FindBin(9.9))/pos_proj.Integral()
        legends[-1].AddEntry(pos_proj, names_flies[iFile]+f" {frac:.2f}", "l")
legends[-1].Draw()
c0.Draw()
#c0.SaveAs("output/ml/ml_conv_dalitz_chi2_ndf_comp_pt76_125.pdf")

In [10]:
pt_min, pt_max = 0.2, 5.
iFile = 1
type_loc = 1
c0 = root.TCanvas(f"c0",f"c0",1450,2*900)
c0.Divide(3,4)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[iFile][type_loc][icentr].Project3D("xy")
    #root.gPad.SetLogz()
    pos_proj.Draw("colz")

c0.Draw()

In [11]:
pt_min, pt_max = 0.4, 5.
type_loc = 0
c0 = root.TCanvas(f"c0",f"c0",1750,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[0][type_loc][icentr].ProjectionX( hists_read[0][type_loc][icentr].GetName()+f"pos_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj = hists_read[2][type_loc][icentr].ProjectionX( hists_read[2][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj1 = hists_read[1][type_loc][icentr].ProjectionX( hists_read[3][type_loc][icentr].GetName()+f"sim_proj1_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj2 = hists_read[3][type_loc][icentr].ProjectionX( hists_read[3][type_loc][icentr].GetName()+f"sim_proj2_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj.Scale(pos_proj.GetMaximum()/(sim_proj.GetMaximum()+1))
    sim_proj1.Scale(pos_proj.GetMaximum()/(sim_proj1.GetMaximum()+1)/3)
    sim_proj2.Scale(pos_proj.GetMaximum()/(sim_proj2.GetMaximum()+1)/1.5)
    Format_Hist_total(pos_proj, title_x="dphi")
    Format_Graph(sim_proj, Lcolor = 2, Lwidth = 2)
    Format_Graph(sim_proj1, Lcolor = root.kGreen+2, Lwidth = 2)
    Format_Graph(sim_proj2, Lcolor = 6, Lwidth = 2)
    pos_proj.Draw("hist")
    sim_proj.Draw("same")
    sim_proj1.Draw("same")
    sim_proj2.Draw("same")
    legends.append(root.TLegend(0.7,0.5,0.95,0.95,all_names[type_loc][icentr]))
    pos_proj.GetXaxis().SetNdivisions(505)
    pos_proj.SetMinimum(0)
    legends[-1].AddEntry(pos_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"conv","l")
    legends[-1].AddEntry(sim_proj1,"dalitz","l")
    legends[-1].AddEntry(sim_proj2,"ccbar","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()

In [12]:
pt_min, pt_max = 0.4, 5.
type_loc = 1
c0 = root.TCanvas(f"c0",f"c0",1450,2*900)
c0.Divide(3,4)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[0][type_loc][icentr].ProjectionX( hists_read[0][type_loc][icentr].GetName()+f"pos_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj = hists_read[1][type_loc][icentr].ProjectionX( hists_read[2][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj1 = hists_read[2][type_loc][icentr].ProjectionX( hists_read[3][type_loc][icentr].GetName()+f"sim_proj1_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj2 = hists_read[3][type_loc][icentr].ProjectionX( hists_read[3][type_loc][icentr].GetName()+f"sim_proj2_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj.Scale(pos_proj.GetMaximum()/(sim_proj.GetMaximum()+1))
    sim_proj1.Scale(pos_proj.GetMaximum()/(sim_proj1.GetMaximum()+1)/3)
    sim_proj2.Scale(pos_proj.GetMaximum()/(sim_proj2.GetMaximum()+1)/1.1)
    Format_Hist_total(pos_proj, title_x="dphi")
    Format_Graph(sim_proj, Lcolor = 2, Lwidth = 2)
    Format_Graph(sim_proj1, Lcolor = 6, Lwidth = 2)
    Format_Graph(sim_proj2, Lcolor = 3, Lwidth = 2)
    pos_proj.Draw("hist")
    sim_proj.Draw("same")
    sim_proj1.Draw("same")
    sim_proj2.Draw("same")
    legends.append(root.TLegend(0.17,0.2,0.95,0.35,all_names[type_loc][icentr]))
    pos_proj.GetXaxis().SetNdivisions(505)
    pos_proj.SetMinimum(0)
    legends[-1].AddEntry(pos_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"dalitz","l")
    legends[-1].AddEntry(sim_proj1,"conv","l")
    legends[-1].AddEntry(sim_proj2,"ccbar","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()

In [13]:
pt_min, pt_max = 1.4, .5
type_loc = 3
iFile = 0
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[iFile][type_loc][icentr].ProjectionX( hists_read[iFile][type_loc][icentr].GetName()+f"pos_proj_{icentr}",1,100,ibinz0,ibinz1)
    second_proj = hists_read[iFile][type_loc-1][icentr].ProjectionX( hists_read[iFile][type_loc-1][icentr].GetName()+f"second_proj_{icentr}",1,100,ibinz0,ibinz1)
    Format_Hist_total(pos_proj, title_x="dphi")
    pos_proj.Draw("hist")
    Format_Graph(second_proj, Lcolor=2, Lwidth=2)
    second_proj.Scale(pos_proj.GetMaximum()/second_proj.GetMaximum())
    second_proj.Draw("hist same")
    legends.append(root.TLegend(0.17,0.2,0.95,0.35,all_names[type_loc][icentr]))
    pos_proj.GetXaxis().SetNdivisions(505)
    pos_proj.SetMinimum(0)
    #legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()

IndexError: list index out of range

In [ ]:
pt_min, pt_max = 0.4, .5
type_loc = 3
c0 = root.TCanvas(f"c0",f"c0",1450,900)
bins = [0.2,0.2,0.2,0.2,0.13]
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[1][type_loc][icentr].ProjectionX( hists_read[1][type_loc][icentr].GetName()+f"pos_proj_{icentr}",1,100,ibinz0,ibinz1)
    second_proj = hists_read[2][type_loc][icentr].ProjectionX( hists_read[2][type_loc-1][icentr].GetName()+f"second_proj_{icentr}",1,100,ibinz0,ibinz1)
    Format_Hist_total(pos_proj, title_x="dphi", title_y="dN/dphi 1/Npi0 * 1e6")
    pos_proj.Draw("hist")
    Format_Graph(second_proj, Lcolor=2, Lwidth=2)
    second_proj.Scale(1./250e6/bins[icentr]*1e6)
    pos_proj.Scale(1.*0.01174/25e6/bins[icentr]*1e6)
    #second_proj.Scale(pos_proj.GetMaximum()/second_proj.GetMaximum())
    pos_proj.SetMaximum(max(pos_proj.GetMaximum(),second_proj.GetMaximum())*1.1)
    second_proj.Draw("hist same")
    legends.append(root.TLegend(0.65,0.6,0.95,0.95,all_names[type_loc][icentr]))
    pos_proj.GetXaxis().SetNdivisions(505)
    pos_proj.SetMinimum(0)
    legends[-1].AddEntry(pos_proj,"dalitz","l")
    legends[-1].AddEntry(second_proj,"conver","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()

In [ ]:
pt_min, pt_max = 0.45, .5
type_loc = 3
c0 = root.TCanvas(f"c0",f"c0",1450,900)
bins = [0.2,0.2,0.2,0.2,0.13]
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[1][type_loc][icentr].ProjectionX( hists_read[1][type_loc][icentr].GetName()+f"pos_proj_{icentr}",1,100,ibinz0,ibinz1)
    second_proj = hists_read[2][type_loc][icentr].ProjectionX( hists_read[2][type_loc-1][icentr].GetName()+f"second_proj_{icentr}",1,100,ibinz0,ibinz1)
    ccbar_proj = hists_read[3][type_loc][icentr].ProjectionX( hists_read[3][type_loc-1][icentr].GetName()+f"second_proj_{icentr}",1,100,ibinz0,ibinz1)
    Format_Hist_total(pos_proj, title_x="dphi", title_y="dN/dphi * 1000")
    pos_proj.Draw("hist")
    Format_Graph(second_proj, Lcolor=2, Lwidth=2)
    Format_Graph(ccbar_proj, Lcolor=6, Lwidth=2)
    second_proj.Scale(1./0.01174/250e6/bins[icentr]*1000)
    pos_proj.Scale(1./25e6/bins[icentr]*1000)
    ccbar_proj.Scale(1./20e6/bins[icentr]*1000)
    #second_proj.Scale(pos_proj.GetMaximum()/second_proj.GetMaximum())
    pos_proj.SetMaximum(max(pos_proj.GetMaximum(),second_proj.GetMaximum(),ccbar_proj.GetMaximum())*1.1)
    second_proj.Draw("hist same")
    ccbar_proj.Draw("hist same")
    legends.append(root.TLegend(0.65,0.6,0.95,0.95,all_names[type_loc][icentr]))
    pos_proj.GetXaxis().SetNdivisions(505)
    pos_proj.SetMinimum(0)
    legends[-1].AddEntry(pos_proj,"dalitz","l")
    legends[-1].AddEntry(second_proj,"conver","l")
    legends[-1].AddEntry(ccbar_proj,"ccbar","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()

In [ ]:
pt_min, pt_max = 0.45, 5.
iFile = 0
type_loc = 4
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[iFile][type_loc][icentr].Project3D("xy")
    #root.gPad.SetLogz()
    pos_proj.Draw("colz")

c0.Draw()

In [ ]:
pt_min, pt_max = 1.2, 2.5
type_loc = 4
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[0][type_loc][icentr].ProjectionX( hists_read[0][type_loc][icentr].GetName()+f"pos_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj = hists_read[1][type_loc][icentr].ProjectionX( hists_read[1][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj1 = hists_read[2][type_loc][icentr].ProjectionX( hists_read[2][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj2 = hists_read[3][type_loc][icentr].ProjectionX( hists_read[3][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    pos_proj.Scale(1./pos_proj.Integral())
    sim_proj.Scale(pos_proj.Integral()/sim_proj.Integral())
    sim_proj1.Scale(pos_proj.Integral()/sim_proj1.Integral())
    sim_proj2.Scale(pos_proj.Integral()/sim_proj2.Integral())
    Format_Hist_total(pos_proj, title_x="dphi", Lwidth=5)
    Format_Graph(sim_proj, Lcolor = 2, Lwidth = 3)
    Format_Graph(sim_proj1, Lcolor = 6, Lwidth = 3)
    Format_Graph(sim_proj2, Lcolor = root.kGreen+2, Lwidth = 3)
    pos_proj.SetMaximum(max(pos_proj.GetMaximum(),sim_proj.GetMaximum(),sim_proj1.GetMaximum(),sim_proj2.GetMaximum())*1.1)
    pos_proj.Draw("hist")
    sim_proj.Draw("same")
    sim_proj1.Draw("same")
    sim_proj2.Draw("same")
    legends.append(root.TLegend(0.67,0.4,0.95,0.95,all_names[type_loc][icentr]))
    pos_proj.GetXaxis().SetNdivisions(505)
    pos_proj.SetMinimum(0)
    legends[-1].AddEntry(pos_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"dalitz","l")
    legends[-1].AddEntry(sim_proj1,"conv","l")
    legends[-1].AddEntry(sim_proj2,"ccbar","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()

In [ ]:
pt_min, pt_max = 1.2, 2.5
type_loc = 4
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projz = hists_read[0][type_loc][0].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(len(hists_read[0][type_loc])):
    c0.cd(1+icentr)
    ibinz0 = projz.FindBin(pt_min)
    ibinz1 = projz.FindBin(pt_max)-1
    pos_proj = hists_read[0][type_loc][icentr].ProjectionY( hists_read[0][type_loc][icentr].GetName()+f"pos_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj = hists_read[1][type_loc][icentr].ProjectionY( hists_read[1][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj1 = hists_read[2][type_loc][icentr].ProjectionY( hists_read[2][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    sim_proj2 = hists_read[3][type_loc][icentr].ProjectionY( hists_read[3][type_loc][icentr].GetName()+f"sim_proj_{icentr}",1,100,ibinz0,ibinz1)
    pos_proj.Scale(1./pos_proj.Integral())
    sim_proj.Scale(1./sim_proj.Integral())
    sim_proj1.Scale(1./sim_proj1.Integral())
    sim_proj2.Scale(1./sim_proj2.Integral())
    Format_Hist_total(pos_proj, title_x="dphi", Lwidth=5)
    Format_Graph(sim_proj, Lcolor = 2, Lwidth = 3)
    Format_Graph(sim_proj1, Lcolor = 6, Lwidth = 3)
    Format_Graph(sim_proj2, Lcolor = root.kGreen+2, Lwidth = 3)
    pos_proj.SetMaximum(max(pos_proj.GetMaximum(),sim_proj.GetMaximum(),sim_proj1.GetMaximum(),sim_proj2.GetMaximum())*1.1)
    pos_proj.Draw("hist")
    sim_proj.Draw("same")
    sim_proj1.Draw("same")
    sim_proj2.Draw("same")
    legends.append(root.TLegend(0.67,0.4,0.95,0.95,all_names[type_loc][icentr]))
    pos_proj.GetXaxis().SetNdivisions(505)
    pos_proj.SetMinimum(0)
    legends[-1].AddEntry(pos_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"dalitz","l")
    legends[-1].AddEntry(sim_proj1,"conv","l")
    legends[-1].AddEntry(sim_proj2,"ccbar","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()